In [1]:
import numpy as np
import cv2
from math import sqrt

In [24]:
# converts the image to black and white
# gets the contours
def preprocess(img):
    # gets the edges of the image
    img = cv2.medianBlur(img,5)
    cimg = cv2.cvtColor(img,cv2.COLOR_GRAY2BGR) # gray image
    canny = cv2.Canny(cimg, 30, 200)
    
    # stores the non-black pixels in a list
    white_pixels = np.nonzero(canny)
    pixels = []
    # an array (1D) of tuples of the x,y pixels that are NOT background
    for i in range(white_pixels[0].shape[0]):
        pixels.append((white_pixels[0][i], white_pixels[1][i]))
    
    return pixels

In [ ]:
# save computation time for square calculation
def pow2(val):
    return val * val

In [27]:
# returns the squared distance between two points (x1,y1), (x2,y2) (not using sqrt to save time)
def distance_square(pt1, pt2):
    return ( pow2(pt1[0] - pt2[0]) + pow2(pt1[1] - pt2[1]) )

In [ ]:
def distance(pt1, pt2):
    return ( sqrt(pow2(pt1[0] - pt2[0]) + pow2(pt1[1] - pt2[1])) )

In [ ]:
def calc_params(pt1, pt2):
    # ellipse's center
    x_0 = (pt1[0] + pt2[0]) / 2
    y_0 = (pt1[1] + pt2[1]) / 2
    
    # half length of major axis
    a = (distance(pt1, pt2)) / 2
    
    # ellipse's orientation
    if pt2[0] == pt1[0]:
        alpha = 0
    else:
        alpha = math.atan( (pt2[1] - pt1[1]) / (pt2[0] - pt1[0]) )
    
    # return center, half length of major axis, and orientation
    return x_0, y_0, a, alpha

In [ ]:
def calc_minor_len(px, pt2, center_pt, a):
    # square distance between (x,y) and (x2,y2)
    f_sq = distance_square(px, pt2)
    
    # square distance between (x,y) and (x0,y0)
    d_sq = distance_square(px, center_pt)
    
    a_sq = pow2(a)
    
    cos_tau = (a_sq + d_sq - f_sq) / (2 * a * sqrt(d_sq))
    sin_tau = 1 - cos_tau
    
    b_sq = (a_sq * d_sq * pow2(sin_tau)) / (a_sq - (d_sq * pow2(cos_tau)))
    
    return sqrt(b_sq)

In [ ]:
'''
ellipse detector function
arguments: pixels array - all the (x,y) pixels in the image that are not the background
           min radius - to ignore ellipse with a radius smaller than this one
'''
def ellipse_detector(pixels_arr, min_r, max_r, vote_count):
    # indices that already were detected, and we don't want to calculate them again
    ignore_indices = set()
    min_r_sq = pow2(min_r)
    max_r_sq = pow2(max_r)
    
    ellipse_detected = [] # store the parameters of the ellipse 
    
    for pixel1 in range(len(pixels_arr)): # step 3           
        for pixel2 in range(pixel1 + 1, len(pixels_arr)): # step 4
            # already tested, move on
            if pixel1 in ignore_indices or pixel2 in ignore_indices:
                continue
            
            # if the (x1,y1), (x2,y2) points are too close or too far don't calculate
            p1_p2_sq_dist = distance_square(pixels_arr[pixel1], pixels_arr[pixel2])
            if(p1_p2_sq_dist < min_r_sq or p1_p2_sq_dist > max_r_sq):  
                continue
            
            # step 5 (assumes ellipse)
            center_x, center_y, major_len, orientation = calc_params(pixel1, pixel2)
            
            accum_arr = []
            
            for pix in range(len(pixels_arr)): # step 6
                if pix == pixel1 or pix == pixel2 or pix in ignore_indices:
                    continue
                
                pix_to_center_dist = distance_square(pixels_arr[pix], (center_x, center_y))
                if pix_to_center_dist < min_r_sq or pix_to_center_dist > max_r_sq: # step 6
                    continue
                    
                # step 7
                minor_len = calc_minor_len(pix, pixel2, (center_x, center_y), major_len)
                
                # the calculated parameters for the ellipse
                cur_ellipse_param = [(center_x, center_y), major_len, minor_len, orientation]
                
                accum_arr.append(cur_ellipse_param) # step 8
            # step 9 - out of this function
            
            # step 10 ??
            # the changes we allow to still be considered as the same ellipse 
#             epsilon = 0.01 # ??
            
            # will be a dict of 
            # key: minor value; value:(count, ellipse_param); -> {minor : (count, ellipse)}
            voters = {}
            
            # go over all the points and count votes for each minor axis
            for e in range(len(accum_arr)):
                curPt = accum_arr[e]
                # the current minor axis length, rounded to 2 dec pt (to avoid too many options)
                curMinorLen = round(curPt[2], 2)
                
                # if the current minor is alrady in the dict, increase the voter count for it
                if curMinorLen in voters: # can do curMinorLen +- epsilon if I want to do it here
                    tmp = voters[curMinorLen]
                    voters[curMinorLen] = (tmp[0]+1, tmp[1])
                    
                    # if we have enough points "voting" for this minor, stop the search
                    if tmp[0]+1 == vote_count:
                        return ellipse_detected.append(tmp[1]) # add the parameters of this ellipse
                
                # else, add current minor to the dict, with the current ellipse parameters
                else:
                    voters[curMinorLen] = (1, curPt) # count is 1, save the current params
                
                
                
                

In [26]:
im = cv2.imread('data/imgs/train/c0/img_104.jpg',0)
i = preprocess(im)

# cv2.imshow('Contours', i)
# cv2.waitKey()
# cv2.destroyAllWindows()

(0, 95)

In [3]:
l = [('abc', 121),('abc', 231),('abc', 148), ('abc',221)]
l.sort(key=lambda x: x[1], reverse=True)
l

[('abc', 231), ('abc', 221), ('abc', 148), ('abc', 121)]

In [15]:
# [(center_x, center_y), major_len, minor_len, orientation]
obj = [(1,2), 1,1,0]
voters = {1 : (10,5)}

a = voters[1]
a[0]
voters[1] = (a[0]+1, a[1])
voters

{1: (11, 5)}